## Imports

In [6]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [2]:
df = pd.read_csv('Sites list (2).xlsx - Sheet1.csv')

In [4]:
df['full_address'] = df['Address'] + ' Manhattan, NY'

## Google Maps API Geolocater Setup

In [7]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [8]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [9]:
df.columns

Index(['Type', 'Address', 'Owner', 'Current Use', 'Zoning',
       'Commercial Overlay', 'Max Units', 'Proposed Units', 'Affordability',
       'MBPO Comments', 'full_address', 'geo_address', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [12]:
df = df.dropna(subset='Address')

In [24]:
df = df.fillna('Not provided')

## HTML popup formatter

In [25]:
def popup_html(row):
    i = row
    Address = df['Address'].iloc[i]
    Type = df['Type'].iloc[i]
    Owner = df['Owner'].iloc[i]
    Current_Use = df['Current Use'].iloc[i]
    Zoning = df['Zoning'].iloc[i]
    Commercial_Overlay = df['Commercial Overlay'].iloc[i]
    Max_Units = df['Max Units'].iloc[i]
    Proposed_Units = df['Proposed Units'].iloc[i]
    Affordability = df['Affordability'].iloc[i]
    Prez_comments = df['MBPO Comments'].iloc[i]
    
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Type: </strong>{}'''.format(Type) + '''<br>
    <strong>Owner: </strong>{}'''.format(Owner) + '''<br>
    <strong>Current Use: </strong>{}'''.format(Current_Use) + '''<br>
    <strong>Zoning: </strong>{}'''.format(Zoning) + '''<br>
    <strong>Commercial Overlay: </strong>{}'''.format(Commercial_Overlay) + '''<br>
    <strong>Max Units: </strong>{}'''.format(Max_Units) + '''<br>
    <strong>Proposed Units: </strong>{}'''.format(Proposed_Units) + '''<br>
    <strong>Affordability: </strong>{}'''.format(Affordability) + '''<br>
    <strong>Comments: </strong>{}'''.format(Prez_comments) + '''
    </html>
    '''
    return html


## Map Maker

In [22]:
import folium
import branca

# f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10.5)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Manhattan Development Opportunities")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=400, max_width=400)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
# m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [20]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [21]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [22]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/multi_family_projects_2023
